# Extended repressilator

This repressilator includes additional "steps" - signalling molecules that act as intermediates between repressors.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1)
tetr = lc.Regulator(name='TetR', degradation_rate=1)
ci = lc.Regulator(name='cI', degradation_rate=1)

# Signalling molecules
c4 = lc.Regulator(name='C4', degradation_rate=1, init_concentration=100)
scb2 = lc.Regulator(name='SCB2', degradation_rate=1)
c14 = lc.Regulator(name='C14', degradation_rate=1)

genetic_n.add_regulator([tetr, laci, ci, c4, scb2, c14])

In [ ]:
# option with producers
# rhli = lc.Regulator(name='RhlI', degradation_rate=1)
# scbb = lc.Regulator(name='ScbB', degradation_rate=1)
# cini = lc.Regulator(name='CinI', degradation_rate=1)
# genetic_n.add_regulator([rhli, scbb, cini])

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
#First extended node
c4_laci = lc.Hill1(name='Prhl', input=c4, output=laci, alpha=[0.1, 10000], K=10, n=2)
laci_not_scb2 = lc.Hill1(name='Plac', input=laci, output=[scb2, mcherry], alpha=[10000,0.1], K=10, n=2)

# Second
scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=tetr, alpha=[0.1,10000], K=10, n=2)
tetr_not_c14 = lc.Hill1(name='Ptet', input=tetr, output=[c14, mven], alpha=[10000,0.1], K=10, n=2)

# Third
c14_ci = lc.Hill1(name='Pcin', input=c14, output=ci, alpha=[0.1, 10000], K=10, n=2)
ci_not_c4 = lc.Hill1(name='PcI', input=ci, output=[c4, cyan], alpha=[10000,0.1], K=10, n=2)


genetic_n.add_operator([c4_laci, laci_not_scb2, scb2_tetr, tetr_not_c14, c14_ci, ci_not_c4])

In [ ]:
# # option with producers

# #First extended node
# c4_laci = lc.Hill1(name='Prhl', input=c4, output=laci, alpha=[0.1, 10000], K=10, n=2)
# laci_not_scbb = lc.Hill1(name='Plac', input=laci, output=[scbb, mcherry], alpha=[10000,0.1], K=10, n=2)
# scbb_scb2 = lc.Producer(name='ScbB', input=scbb, output=scb2, k1a=1, k2=1)

# # Second
# scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=tetr, alpha=[0.1,10000], K=10, n=2)
# tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[cini, mven], alpha=[10000,0.1], K=10, n=2)
# cini_c14 = lc.Producer(name='CinI', input=cini, output=c14, k1a=1, k2=1)

# # Third
# c14_ci = lc.Hill1(name='Pcin', input=c14, output=ci, alpha=[0.1, 10000], K=10, n=2)
# ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=[rhli, cyan], alpha=[10000,0.1], K=10, n=2)
# rhli_c4 = lc.Producer(name='ScbB', input=rhli, output=c4, k1a=1, k2=1)


# genetic_n.add_operator([c4_laci, laci_not_scbb, scbb_scb2, scb2_tetr, tetr_not_cini, cini_c14, c14_ci, ci_not_rhli, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

Then I can also change how each signal diffuses (to see how that affects the system)